# Import to AWS DB

1) Run `CalculateEverything` in the `InternationalityIndex.InternationalityCalculations.py`

2) Copy the output xlsx file in the same folder as this notebook.

3) Edit the first two rows in the following cell and run it.

4) Copy all files from the AWS_Import directory using WinSCP

    a. Connect to AWS EC2 IDEA (ubuntu@ec2-18-188-88-0.us-east-2.compute.amazonaws.com)
    
    b. Copy csv files from the `AWS_Import` directory to `\home\ubuntu\db-admin\csv`
    
5) Using Putty, run the import to AWS

    a. Connect to AWS EC2 IDEA (ubuntu@ec2-18-188-88-0.us-east-2.compute.amazonaws.com)
    
    b. Go to `db-admin` directory
    
    c. run: `psql --host=science-internationality-dbinstance.c3aa5fkeiz2h.us-east-2.rds.amazonaws.com --port=5432 --username=root --password --dbname=scienceInternationalitydb -f drop_generate_schema.sql`
    
    d. run: `psql --host=science-internationality-dbinstance.c3aa5fkeiz2h.us-east-2.rds.amazonaws.com --port=5432 --username=root --password --dbname=scienceInternationalitydb -f psql-import-csvs.txt`
    
    
In case of problems check
    a. Variable names - from the original excel in additionalData, through the table schema in drop_generate_schema.sql to variable names in psql-import-csvs.txt
    
    b. Data validity in CSVs.
    
    c. Also prisma query in fetcher.js should contain valid variable names! If they change, prisma should be rerun as follows:
        1. docker-compose down
        2. change the datamodel.yml
        3. docker-compose up -d prisma
        4. prisma deploy

In [ ]:
topData = '20181106_AllFieldsCountriesMethods_TOP.xlsx' ## OUTPUT OF CalculateEverything() in InternationalityIndex.InternationalityCalculations.py
bottomData = '20181128_AllFieldsCountriesMethods_bot.xlsx'
additionalData = 'populateAmazon.xlsx'

import pandas as pd

# load data
tops = pd.read_excel(topData,index_col=[0,1,2,3]).reset_index()
bottoms = pd.read_excel(bottomData,index_col=[0,1,2,3]).reset_index()
bottoms = bottoms[bottoms.Field != 'All']
df = pd.concat([tops,bottoms],ignore_index=True)

countries = pd.read_excel(additionalData,sheet_name='country')
index = pd.read_excel(additionalData,sheet_name='index',index=False)

merged = pd.merge(df,countries,how='left',left_on='Country',right_on='name').loc[:,['country_code','Field','Method','Period','Internationality']]
merged.columns = index.columns

## filter displayed globalizations

In [ ]:
import pandas as pd
import sqlite3
import numpy as np

jrnThreshold = 30
fields = merged.field_code.unique()


def getDocsJournalsForField(field,conn):

    if field == 'All':
        query ='''
        SELECT
       c.name as Country,
       p.name as Year,
       Sum(A.Articles) AS Documents,
       Count(A.Articles) as Journals

        FROM ArticleCountries as A
        INNER JOIN countries c on A.FacetID = c.ID
        INNER JOIN periods p on A.PeriodID = p.ID
        INNER JOIN issns i on A.ISSNID = i.ID
        GROUP BY Country,Year

        '''
    else:
        query = '''
        SELECT
       c.name as Country,
       p.name as Year,
       Sum(A.Articles) AS Documents,
       Count(A.Articles) as Journals

        FROM ArticleCountries as A
        INNER JOIN countries c on A.FacetID = c.ID
        INNER JOIN periods p on A.PeriodID = p.ID
        INNER JOIN issns i on A.ISSNID = i.ID
        WHERE i.{} = 1
        GROUP BY Country,Year
        
        '''.format(field)

    df = pd.read_sql_query(query,conn)
    df['field'] = field
    return df

#conn = sqlite3.connect('D:/Dropbox/Python/AllScopusJournals/180802_1611_AllJournals_ArReCp_2001_2017.sqlite')
conn = sqlite3.connect('C:/Users/vitekzkytek/Dropbox/Python/AllScopusJournals/180802_1611_AllJournals_ArReCp_2001_2017.sqlite')
dfs =[]

for field in fields:
    dfs.append(getDocsJournalsForField(field,conn))

filters = pd.concat(dfs).merge(countries.loc[:,['country_code','name']],left_on='Country',right_on='name').drop('name',axis=1)
filters['include'] = np.where(filters['Journals'] >= jrnThreshold, True, False)
filters.Year = pd.to_numeric(filters.Year)


In [ ]:
merged = merged.merge(filters,left_on=['country_code','field_code','period'],right_on=['country_code','field','Year'],how='left')
merged = merged[merged['include'] == True]
merged = merged.drop(['Documents','Journals','field','Country','Year','include'],axis=1)

## Calculate group averages

In [ ]:
avgs = [merged]
def calcGroupAverage(mergedDF,countriesDF,dimension,new_country_codes):
    df = mergedDF.merge(countriesDF,on='country_code',how='left').set_index(keys=['country_code','field_code','method_code','period'])
    df = df[['value',dimension]]
    g = df.groupby(['field_code','method_code','period',dimension]).mean().reset_index()
    g['country_code'] = g[dimension].map(new_country_codes,na_action='ignore')
    return g.drop(dimension,axis=1)[mergedDF.columns].dropna()

cntrs = [countries]
def appendToCountries(countries,d):
    l = [{'country_code':d[key],'name':key,'Type':'aggregate'} for key in d.keys()]
    df = pd.DataFrame(l)
    return df

#regions
d = {'North America':'_NAmer',
     'East Asia':'_EAsia',
     'Europe':'_Europe',
     'South Asia':'_SAsia',
     'Pacific':'_Pac',
     'South America':'_SAmer',
     'Central Asia':'_CAsia',
     'Middle East':'_MEast',
     'Sub-Saharan Africa':'_SSAfr',
     'North Africa':'_NAfr'
    }
avgs.append(calcGroupAverage(merged,countries, 'region',d))
cntrs.append(appendToCountries(countries,d))


# Income Level
d = {'Upper middle income':'_UMI','High income':'_HI','Lower middle income':'_LMI','Low income':'_LI'}
avgs.append(calcGroupAverage(merged,countries, 'incomelevel',d))
cntrs.append(appendToCountries(countries,d))


#EU
d = {'EU-15':'_EU15','EU-13':'_EU13'}
avgs.append(calcGroupAverage(merged,countries, 'eu_sub',d))
cntrs.append(appendToCountries(countries,d))

# whole EU
d = {'EU':'_EU'}
avgs.append(calcGroupAverage(merged,countries, 'eu',d))
cntrs.append(appendToCountries(countries,d))


#OECD
d = {'OECD':'_OECD'}
avgs.append(calcGroupAverage(merged,countries, 'oecd',d))
cntrs.append(appendToCountries(countries,d))


#IMF 2003
d = {
    'Advanced Countries':'_ADV',
    'Developing Countries':'_DEV',
    'Transition Countries':'_TRA'
}
avgs.append(calcGroupAverage(merged,countries, 'imf2003',d))
cntrs.append(appendToCountries(countries,d))

# World
world = merged.set_index(['country_code','field_code','method_code','period']).unstack('country_code').mean(axis=1).rename('value').reset_index()
world['country_code'] = '_AV'
world = world[merged.columns]
avgs.append(world)
cntrs.append(appendToCountries(countries,{'World':'_AV'}))

merged = pd.concat(avgs,ignore_index=True)
countries = pd.concat(cntrs,ignore_index=True)

### Write CSVs

In [ ]:
countries.to_csv('AWS_Import/country.csv',index=False)

method = pd.read_excel(additionalData,sheet_name='method').to_csv('AWS_Import/method.csv',index=False)
field = pd.read_excel(additionalData,sheet_name='field').to_csv('AWS_Import/field.csv',index=False)
merged.to_csv('AWS_Import/index.csv',index=False)

# Generate JSONs with controlling data

1. Run both cells in the notebook

2. Copy the file `controls_data.js` from this notebooks directory into `main/public/javascripts/` directory

In [ ]:
import pandas as pd
df_methods = pd.read_excel('populateAmazon.xlsx',sheet_name='method',index_col='method_code')
df_methods = df_methods.loc[['euclid','weightGini','localShare','shareEnglish','top3','GiniSimpson'],:]
df_fields = pd.read_excel('populateAmazon.xlsx',sheet_name='field',index_col='field_code')
df_countries = pd.read_excel('populateAmazon.xlsx',sheet_name='country',index_col='country_code')
df_countries = countries.set_index('country_code')[df_countries.columns]

In [ ]:
d_aggr = df_countries.loc[df_countries.Type == 'aggregate','name'].reset_index().rename(columns={'country_code':'id','name':'text'}).to_dict(orient='records')
d_cntrs = df_countries.loc[df_countries.Type == 'country','name'].reset_index().rename(columns={'country_code':'id','name':'text'}).sort_values('text').to_dict(orient='records')
d_countries = {'results':[{'text':'Country Groups','children':d_aggr},{'text':'Countries','children':d_cntrs}],'pagination':{'more':True}}

d_tops = df_fields.loc[df_fields.level == 'TOP','name'].reset_index().rename(columns={'field_code':'id','name':'text'}).to_dict(orient='records')
d_bottoms = df_fields.loc[df_fields.level == 'BOT','name'].reset_index().rename(columns={'field_code':'id','name':'text'}).to_dict(orient='records')
d_fields = {'results':[{'text':'Broad Subject clusters','children':d_tops},{'text':'Major Subject Areas','children':d_bottoms}],'pagination':{'more':True}}

d_methods = df_methods.reset_index().rename(columns={'method_code':'id','name':'text'}).to_dict(orient='records')
d_methods = {'results':d_methods,'pagination':{'more':True}}

d = {'methods': d_methods,'fields':d_fields,'countries':d_countries}

import json
s = 'var controllers = %s' % (json.dumps(d))

with open("controls_data.js", "w") as f:
    f.write(s)